In [ ]:
"""entonces es simplemente dado un estado y lo que se hizo en el estado para aumentar 
el score , luego es hacer factor comunt entre los estados y sus outputs para crear una
estructura equivalente.Pero entonces como hace para saber cual output seleccionar en
primer lieu ? c'est aleatoire s'il ne connait pas l'etat actuel ,apres tu pourra faire
la comparaison d'un nouvel etat par rapport a un autre connu en comparant le nombre de 
bits egaux de l'etat actuel"""

In [2]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [14]:

import pygame
import random

# Constants
GRID_SIZE = 20
MATRIX_WIDTH = 40
MATRIX_HEIGHT = 15
WIDTH, HEIGHT = 800, 600
MATRIX_HEIGHT_PX = HEIGHT // 2

# Initialize pygame
pygame.init()

# Screen setup
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Game and Matrix Window")

# Colors
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BRIGHT_GREEN = (150, 255, 150)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)

# Game objects
PLAYER_WIDTH, PLAYER_HEIGHT = 40, 40
FOOD_SIZE = 5
player1 = pygame.Rect(WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)
player2 = pygame.Rect(3 * WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)

# Food system
class Food:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, FOOD_SIZE, FOOD_SIZE)

food_list = [Food(random.randint(0, WIDTH - FOOD_SIZE), 
                 random.randint(0, HEIGHT // 2 - FOOD_SIZE)) for _ in range(5)]

# Player states
player1_life = 10
player2_life = 10
blue_last_state = None
blue_last_action = None
blue_last_reward = None
blue_experiences = []
blue_saved_states = set()  # Track unique states blue has encountered

# Fonts
font = pygame.font.Font(None, 36)

def spawn_new_food():
    """Spawn new food ensuring no overlap with players."""
    while True:
        x = random.randint(0, WIDTH - FOOD_SIZE)
        y = random.randint(0, HEIGHT // 2 - FOOD_SIZE)
        new_food = pygame.Rect(x, y, FOOD_SIZE, FOOD_SIZE)
        # Ensure food doesn't overlap players
        if not player1.colliderect(new_food) and not player2.colliderect(new_food):
            return Food(x, y)

def update_matrix():
    game_matrix = [[0] * MATRIX_WIDTH for _ in range(MATRIX_HEIGHT)]
    for player in [player1, player2]:
        for y in range(player.top // GRID_SIZE, (player.bottom) // GRID_SIZE):
            for x in range(player.left // GRID_SIZE, (player.right) // GRID_SIZE):
                if 0 <= y < MATRIX_HEIGHT and 0 <= x < MATRIX_WIDTH:
                    game_matrix[y][x] = 1
    for food in food_list:
        row, col = food.rect.top // GRID_SIZE, food.rect.left // GRID_SIZE
        if 0 <= row < MATRIX_HEIGHT and 0 <= col < MATRIX_WIDTH:
            game_matrix[row][col] = 1
    return game_matrix

def draw_matrix(game_matrix):
    matrix_surface = pygame.Surface((WIDTH, MATRIX_HEIGHT_PX))
    matrix_surface.fill(BLACK)
    for y, row in enumerate(game_matrix):
        row_text = " ".join(str(cell) for cell in row)
        text_surface = font.render(row_text, True, WHITE)
        matrix_surface.blit(text_surface, (10, y * 20))
    screen.blit(matrix_surface, (0, HEIGHT // 2))

def draw_game():
    screen.fill(BLACK)
    
    # Draw players
    pygame.draw.rect(screen, BLUE, player1)
    pygame.draw.rect(screen, RED, player2)
    
    # Draw food
    for food in food_list:
        pygame.draw.rect(screen, GREEN, food.rect)
    
    life_text1 = font.render(f"Player 1 Life: {player1_life}", True, WHITE)
    life_text2 = font.render(f"Player 2 Life: {player2_life}", True, WHITE)
    screen.blit(life_text1, (10, 10))
    screen.blit(life_text2, (WIDTH - 200, 10))

def handle_movement():
    global player2
    keys = pygame.key.get_pressed()
    move_speed = 5

    # Player 2 movement
    if keys[pygame.K_UP] and player2.top > 0 and not player1.colliderect(player2.move(0, -move_speed)) and not food_collision(player2.move(0, -move_speed)):
        player2.y -= move_speed
    if keys[pygame.K_DOWN] and player2.bottom < HEIGHT // 2 and not player1.colliderect(player2.move(0, move_speed)) and not food_collision(player2.move(0, move_speed)):
        player2.y += move_speed
    if keys[pygame.K_LEFT] and player2.left > 0 and not player1.colliderect(player2.move(-move_speed, 0)) and not food_collision(player2.move(-move_speed, 0)):
        player2.x -= move_speed
    if keys[pygame.K_RIGHT] and player2.right < WIDTH and not player1.colliderect(player2.move(move_speed, 0)) and not food_collision(player2.move(move_speed, 0)):
        player2.x += move_speed

def food_collision(player):
    """Check if a player collides with any food."""
    for food in food_list:
        if player.colliderect(food.rect):
            return True
    return False

def blue_player_logic(game_matrix):
    global player1_life, player2_life, blue_last_state, blue_last_action, blue_last_reward, blue_saved_states
    actions = ["UP", "DOWN", "LEFT", "RIGHT", "EAT", "ATTACK"]
    
    # Define the size of the surrounding area to focus on (e.g., a 3x3 grid)
    surrounding_area_size = 3  # This defines a 3x3 grid around the player
    x_start = max(0, player1.left // GRID_SIZE - surrounding_area_size // 2)
    x_end = min(MATRIX_WIDTH, player1.right // GRID_SIZE + surrounding_area_size // 2)
    y_start = max(0, player1.top // GRID_SIZE - surrounding_area_size // 2)
    y_end = min(MATRIX_HEIGHT, player1.bottom // GRID_SIZE + surrounding_area_size // 2)

    # Extract the relevant portion of the game matrix to form the state
    blue_last_state = []
    for y in range(y_start, y_end):
        row = game_matrix[y][x_start:x_end]
        blue_last_state.append(row)

    # Save the state (convert to tuple to ensure it's hashable)
    blue_saved_states.add(tuple(map(tuple, blue_last_state)))  # Add state to saved states (as a tuple)

    # Calculate reward based on life difference
    reward = player1_life - blue_last_reward if blue_last_reward is not None else 0
    blue_experiences.append((blue_last_state, blue_last_action, reward))
    blue_last_reward = player1_life

    # Choose a random action (can now be a combination)
    blue_last_action = random.sample(actions, k=random.randint(1, 3))  # Random combo of 1-3 actions

    # Perform the actions
    if "UP" in blue_last_action and player1.top > 0:
        player1.y -= 5
    if "DOWN" in blue_last_action and player1.bottom < HEIGHT // 2:
        player1.y += 5
    if "LEFT" in blue_last_action and player1.left > 0:
        player1.x -= 5
    if "RIGHT" in blue_last_action and player1.right < WIDTH:
        player1.x += 5
    if "EAT" in blue_last_action:
        # Check if player1 is near food and eat it
        for food in food_list[:]:
            if player1.colliderect(food.rect):
                food_list.remove(food)
                player1_life += 1
                food_list.append(spawn_new_food())
    if "ATTACK" in blue_last_action:
        # Check if player1 is near player2 and attack
        if player1.colliderect(player2):
            player2_life -= 1  # Attack reduces life of player 2

def handle_food():
    global food_list, player1_life, player2_life
    
    for food in food_list[:]:
        if player1.colliderect(food.rect):
            food_list.remove(food)
            player1_life += 1
            food_list.append(spawn_new_food())
            
        if player2.colliderect(food.rect):
            food_list.remove(food)
            player2_life += 1
            food_list.append(spawn_new_food())

def display_debug_info():
    """Display the blue player's state, action, and reward in real-time."""
    debug_area = pygame.Surface((WIDTH, 100))
    debug_area.fill((30, 30, 30))
    
    state_text = font.render(f"State: {blue_last_state}", True, WHITE)  # Display surrounding state
    action_text = font.render(f"Action: {blue_last_action}", True, WHITE)
    reward_text = font.render(f"Reward: {blue_last_reward}", True, WHITE)
    saved_states_text = font.render(f"Saved States: {len(blue_saved_states)}", True, WHITE)
    
    debug_area.blit(state_text, (10, 10))
    debug_area.blit(action_text, (10, 40))
    debug_area.blit(reward_text, (10, 70))
    debug_area.blit(saved_states_text, (10, 100))  # Show the count of unique saved states
    screen.blit(debug_area, (0, HEIGHT - 100))

# Game loop
clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    handle_movement()
    game_matrix = update_matrix()
    blue_player_logic(game_matrix)
    handle_food()
    
    draw_game()
    draw_matrix(game_matrix)
    display_debug_info()

    pygame.display.update()
    clock.tick(60)

pygame.quit()


In [18]:
import pygame
import random

# Constants
GRID_SIZE = 20
MATRIX_WIDTH = 40
MATRIX_HEIGHT = 15
WIDTH, HEIGHT = 800, 600
MATRIX_HEIGHT_PX = HEIGHT // 2

# Initialize pygame
pygame.init()

# Screen setup
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Game and Matrix Window")

# Colors
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BRIGHT_GREEN = (150, 255, 150)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)

# Game objects
PLAYER_WIDTH, PLAYER_HEIGHT = 40, 40
FOOD_SIZE = 5
player1 = pygame.Rect(WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)
player2 = pygame.Rect(3 * WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)

# Food system
class Food:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, FOOD_SIZE, FOOD_SIZE)

food_list = [Food(random.randint(0, WIDTH - FOOD_SIZE), 
                 random.randint(0, HEIGHT // 2 - FOOD_SIZE)) for _ in range(5)]

# Player states
player1_life = 10
player2_life = 10
blue_last_state = None
blue_last_action = None
blue_last_reward = None
blue_experiences = []
blue_saved_states = set()  # Track unique states blue has encountered

# Fonts
font = pygame.font.Font(None, 36)

def spawn_new_food():
    """Spawn new food ensuring no overlap with players."""
    while True:
        x = random.randint(0, WIDTH - FOOD_SIZE)
        y = random.randint(0, HEIGHT // 2 - FOOD_SIZE)
        new_food = pygame.Rect(x, y, FOOD_SIZE, FOOD_SIZE)
        if not player1.colliderect(new_food) and not player2.colliderect(new_food):
            return Food(x, y)

def update_matrix():
    game_matrix = [[0] * MATRIX_WIDTH for _ in range(MATRIX_HEIGHT)]
    for player in [player1, player2]:
        for y in range(player.top // GRID_SIZE, (player.bottom) // GRID_SIZE):
            for x in range(player.left // GRID_SIZE, (player.right) // GRID_SIZE):
                if 0 <= y < MATRIX_HEIGHT and 0 <= x < MATRIX_WIDTH:
                    game_matrix[y][x] = 1
    for food in food_list:
        row, col = food.rect.top // GRID_SIZE, food.rect.left // GRID_SIZE
        if 0 <= row < MATRIX_HEIGHT and 0 <= col < MATRIX_WIDTH:
            game_matrix[row][col] = 1
    return game_matrix

def draw_matrix(game_matrix):
    matrix_surface = pygame.Surface((WIDTH, MATRIX_HEIGHT_PX))
    matrix_surface.fill(BLACK)
    for y, row in enumerate(game_matrix):
        row_text = " ".join(str(cell) for cell in row)
        text_surface = font.render(row_text, True, WHITE)
        matrix_surface.blit(text_surface, (10, y * 20))
    screen.blit(matrix_surface, (0, HEIGHT // 2))

def draw_game():
    screen.fill(BLACK)
    pygame.draw.rect(screen, BLUE, player1)
    pygame.draw.rect(screen, RED, player2)
    for food in food_list:
        pygame.draw.rect(screen, GREEN, food.rect)
    life_text1 = font.render(f"Player 1 Life: {player1_life}", True, WHITE)
    life_text2 = font.render(f"Player 2 Life: {player2_life}", True, WHITE)
    screen.blit(life_text1, (10, 10))
    screen.blit(life_text2, (WIDTH - 200, 10))

def handle_movement():
    global player2, player2_life, food_list
    keys = pygame.key.get_pressed()
    move_speed = 5

    # Player 2 movement
    if keys[pygame.K_UP] and player2.top > 0:
        player2.y -= move_speed
    if keys[pygame.K_DOWN] and player2.bottom < HEIGHT // 2:
        player2.y += move_speed
    if keys[pygame.K_LEFT] and player2.left > 0:
        player2.x -= move_speed
    if keys[pygame.K_RIGHT] and player2.right < WIDTH:
        player2.x += move_speed

    # Check for food collisions and "eat" the food
    for food in food_list[:]:  # Iterate through a copy of the food list
        if player2.colliderect(food.rect):
            food_list.remove(food)  # Remove the food from the list
            player2_life += 1  # Increase player2's life
            food_list.append(spawn_new_food())  # Spawn a new food

def food_collision(player):
    """Check if a player collides with any food."""
    for food in food_list:
        if player.colliderect(food.rect):
            return True
    return False

def blue_player_logic(game_matrix):
    global player1_life, player2_life, blue_last_state, blue_last_action, blue_last_reward, blue_saved_states
    actions = ["UP", "DOWN", "LEFT", "RIGHT", "EAT", "ATTACK"]
    surrounding_area_size = 3
    x_start = max(0, player1.left // GRID_SIZE - surrounding_area_size // 2)
    x_end = min(MATRIX_WIDTH, player1.right // GRID_SIZE + surrounding_area_size // 2)
    y_start = max(0, player1.top // GRID_SIZE - surrounding_area_size // 2)
    y_end = min(MATRIX_HEIGHT, player1.bottom // GRID_SIZE + surrounding_area_size // 2)

    blue_last_state = []
    for y in range(y_start, y_end):
        row = game_matrix[y][x_start:x_end]
        blue_last_state.append(row)

    blue_saved_states.add(tuple(map(tuple, blue_last_state)))

    reward = player1_life - blue_last_reward if blue_last_reward is not None else 0
    blue_experiences.append((blue_last_state, blue_last_action, reward))
    blue_last_reward = player1_life

    blue_last_action = random.sample(actions, k=random.randint(1, 3))
    if "UP" in blue_last_action and player1.top > 0:
        player1.y -= 5
    if "DOWN" in blue_last_action and player1.bottom < HEIGHT // 2:
        player1.y += 5
    if "LEFT" in blue_last_action and player1.left > 0:
        player1.x -= 5
    if "RIGHT" in blue_last_action and player1.right < WIDTH:
        player1.x += 5
    if "EAT" in blue_last_action:
        for food in food_list[:]:
            if player1.colliderect(food.rect):
                food_list.remove(food)
                player1_life += 1
                food_list.append(spawn_new_food())
    if "ATTACK" in blue_last_action:
        if player1.colliderect(player2):
            player2_life -= 1

def display_debug_info():
    debug_area = pygame.Surface((WIDTH, 100))
    debug_area.fill((30, 30, 30))
    state_text = font.render(f"State: {blue_last_state}", True, WHITE)
    action_text = font.render(f"Action: {blue_last_action}", True, WHITE)
    reward_text = font.render(f"Reward: {blue_last_reward}", True, WHITE)
    saved_states_text = font.render(f"Saved States: {len(blue_saved_states)}", True, WHITE)
    debug_area.blit(state_text, (10, 10))
    debug_area.blit(action_text, (10, 40))
    debug_area.blit(reward_text, (10, 70))
    debug_area.blit(saved_states_text, (10, 100))
    screen.blit(debug_area, (0, HEIGHT - 100))

# Game loop
clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    handle_movement()
    game_matrix = update_matrix()
    blue_player_logic(game_matrix)
    draw_game()
    draw_matrix(game_matrix)
    display_debug_info()
    pygame.display.update()
    clock.tick(60)

pygame.quit()


In [21]:
import pygame
import random

# Constants
GRID_SIZE = 20
MATRIX_WIDTH = 40
MATRIX_HEIGHT = 15
WIDTH, HEIGHT = 800, 600
MATRIX_HEIGHT_PX = HEIGHT // 2

# Initialize pygame
pygame.init()

# Screen setup
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Game and Matrix Window")

# Colors
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)

# Game objects
PLAYER_WIDTH, PLAYER_HEIGHT = 40, 40
FOOD_SIZE = 5
player1 = pygame.Rect(WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)
player2 = pygame.Rect(3 * WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)

# Food system
class Food:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, FOOD_SIZE, FOOD_SIZE)

food_list = [Food(random.randint(0, WIDTH - FOOD_SIZE), 
                 random.randint(0, HEIGHT // 2 - FOOD_SIZE)) for _ in range(5)]

# Player states
player1_life = 10
player2_life = 10

# Fonts
font = pygame.font.Font(None, 36)

def spawn_new_food():
    """Spawn new food ensuring no overlap with players."""
    while True:
        x = random.randint(0, WIDTH - FOOD_SIZE)
        y = random.randint(0, HEIGHT // 2 - FOOD_SIZE)
        new_food = pygame.Rect(x, y, FOOD_SIZE, FOOD_SIZE)
        if not player1.colliderect(new_food) and not player2.colliderect(new_food):
            return Food(x, y)

def update_matrix():
    """Update the matrix with players and food positions."""
    game_matrix = [[0] * MATRIX_WIDTH for _ in range(MATRIX_HEIGHT)]
    
    # Mark player positions
    for player in [player1, player2]:
        for y in range(player.top // GRID_SIZE, (player.bottom) // GRID_SIZE):
            for x in range(player.left // GRID_SIZE, (player.right) // GRID_SIZE):
                if 0 <= y < MATRIX_HEIGHT and 0 <= x < MATRIX_WIDTH:
                    game_matrix[y][x] = 1
    
    # Mark food positions
    for food in food_list:
        row, col = food.rect.top // GRID_SIZE, food.rect.left // GRID_SIZE
        if 0 <= row < MATRIX_HEIGHT and 0 <= col < MATRIX_WIDTH:
            game_matrix[row][col] = 1
    
    return game_matrix

def draw_matrix(game_matrix):
    """Render the matrix as text on the screen."""
    matrix_surface = pygame.Surface((WIDTH, MATRIX_HEIGHT_PX))
    matrix_surface.fill(BLACK)
    for y, row in enumerate(game_matrix):
        row_text = " ".join(str(cell) for cell in row)
        text_surface = font.render(row_text, True, WHITE)
        matrix_surface.blit(text_surface, (10, y * 20))
    screen.blit(matrix_surface, (0, HEIGHT // 2))

def draw_game():
    """Render the game view."""
    screen.fill(BLACK)
    pygame.draw.rect(screen, BLUE, player1)
    pygame.draw.rect(screen, RED, player2)
    for food in food_list:
        pygame.draw.rect(screen, GREEN, food.rect)
    life_text1 = font.render(f"Player 1 Life: {player1_life}", True, WHITE)
    life_text2 = font.render(f"Player 2 Life: {player2_life}", True, WHITE)
    screen.blit(life_text1, (10, 10))
    screen.blit(life_text2, (WIDTH - 200, 10))

def handle_movement():
    """Handle movement and prevent overlap."""
    global player2
    keys = pygame.key.get_pressed()
    move_speed = 5

    # Player 2 movement
    if keys[pygame.K_UP] and player2.top > 0:
        player2.y -= move_speed
    if keys[pygame.K_DOWN] and player2.bottom < HEIGHT // 2:
        player2.y += move_speed
    if keys[pygame.K_LEFT] and player2.left > 0:
        player2.x -= move_speed
    if keys[pygame.K_RIGHT] and player2.right < WIDTH:
        player2.x += move_speed

    # Prevent overlapping
    if player1.colliderect(player2):
        if keys[pygame.K_LEFT]:
            player2.x += move_speed
        if keys[pygame.K_RIGHT]:
            player2.x -= move_speed
        if keys[pygame.K_UP]:
            player2.y += move_speed
        if keys[pygame.K_DOWN]:
            player2.y -= move_speed

def player2_eat():
    """Player 2 eats when pressing 'M'."""
    global player2_life
    keys = pygame.key.get_pressed()
    if keys[pygame.K_m]:
        for food in food_list[:]:
            if player2.colliderect(food.rect):
                food_list.remove(food)
                player2_life += 1
                food_list.append(spawn_new_food())
                break

def blue_player_logic():
    """Blue player logic with EAT command."""
    global player1_life
    actions = ["UP", "DOWN", "LEFT", "RIGHT", "EAT"]
    action = random.choice(actions)

    if action == "UP" and player1.top > 0:
        player1.y -= 5
    if action == "DOWN" and player1.bottom < HEIGHT // 2:
        player1.y += 5
    if action == "LEFT" and player1.left > 0:
        player1.x -= 5
    if action == "RIGHT" and player1.right < WIDTH:
        player1.x += 5
    if action == "EAT":
        for food in food_list[:]:
            if player1.colliderect(food.rect):
                food_list.remove(food)
                player1_life += 1
                food_list.append(spawn_new_food())
                break

# Game loop
clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    handle_movement()
    player2_eat()
    game_matrix = update_matrix()
    blue_player_logic()
    draw_game()
    draw_matrix(game_matrix)
    pygame.display.update()
    clock.tick(60)

pygame.quit()
